# Deep Q-Learning

In this notebook, you will implement a deep Q-Learning reinforcement algorithm. The implementation borrows ideas from both the original DeepMind Nature paper and the more recent asynchronous version:<br/>
[1] "Human-Level Control through Deep Reinforcement Learning" by Mnih et al. 2015<br/>
[2] "Asynchronous Methods for Deep Reinforcement Learning" by Mnih et al. 2016.<br/>

In particular:
* We use separate target and Q-functions estimators with periodic updates to the target estimator. 
* We use several concurrent "threads" rather than experience replay to generate less biased gradient updates. 
* Threads are actually synchronized so we start each one at a random number of moves.
* We use an epsilon-greedy policy that blends random moves with policy moves.
* We taper the random action parameter (epsilon) and the learning rate to zero during training.

This gives a simple and reasonably fast general-purpose RL algorithm. We use it here for the Cartpole environment from OpenAI Gym, but it can easily be adapted to others. For this notebook, you will implement 4 steps:

1. The backward step for the Q-estimator
2. The $\epsilon$-greedy policy
3. "asynchronous" initialization 
4. The Q-learning algorithm

To get started, we import some prerequisites.

In [1]:
%matplotlib inline

import gym
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
import pickle

/Users/rajagrawal/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


The block below lists some parameters you can tune. They should be self-explanatory. They are currently set to train CartPole-V0 to a "solved" score (> 195) most of the time. 

In [2]:
nsteps = 10001                       # Number of steps to run (game actions per environment)
npar = 16                            # Number of parallel environments
target_window = 200                  # Interval to update target estimator from q-estimator
discount_factor = 0.99               # Reward discount factor
printsteps = 1000                    # Number of steps between printouts
render = False                       # Whether to render an environment while training

epsilon_start = 1.0                  # Parameters for epsilon-greedy policy: initial epsilon
epsilon_end = 0.0                    # Final epsilon
neps = int(0.8*nsteps)               # Number of steps to decay epsilon

learning_rate = 2e-3                 # Initial learning rate
lr_end = 0                           # Final learning rate
nlr = neps                           # Steps to decay learning rate
decay_rate = 0.99                    # Decay factor for RMSProp 

nhidden = 200                        # Number of hidden layers for estimators

init_moves = 2000                    # Upper bound on random number of moves to take initially
nwindow = 2                          # Sensing window = last n images in a state

Below are environment-specific parameters. The function "preprocess" should process an observation returned by the environment into a vector for training. For CartPole we simply append a 1 to implement bias in the first layer. 

For visual environments you would typically crop, downsample to 80x80, set color to a single bit (foreground/background), and flatten to a vector. That transformation is already implemented in the Policy Gradient code.

*nfeats* is the dimension of the vector output by *preprocess*.

In [3]:
game_type="CartPole-v0"                 # Model type and action definitions
VALID_ACTIONS = [0, 1]
nactions = len(VALID_ACTIONS)
nfeats = 5                              # There are four state features plus the constant we add

def preprocess(I):                      # preprocess each observation
    """Just append a 1 to the end"""
    return np.append(I.astype(float),1) # Add a constant feature for bias

Here is the Q-estimator class. We use two instances of this class, one for the target estimator, and one for the Q-estimator. The Q function is normally represented as a scalar $Q(x,a)$ where $x$ is the state and $a$ is an action. For ease of implementation, we actually estimate a vector-valued function $Q(x,.)$ which returns the estimated reward for every action. The model here has just a single hidden layer:

<pre>
Input Layer (nfeats) => FC Layer => RELU => FC Layer => Output (naction values)
</pre>

## 1. Implement Q-estimator gradient
Your first task is to implement the
<pre>Estimator.gradient(s, a, y)</pre>
method for this class. **gradient** should compute the gradients wrt weight arrays W1 and W2 into
<pre>self.grad['W1']
self.grad['W2']</pre>
respectively. Both <code>a</code> and <code>y</code> are vectors. Be sure to update only the output layer weights corresponding to the given action vector. 

In [4]:
class Estimator():

    def __init__(self, ninputs, nhidden, nactions):
        """ Create model matrices, and gradient and squared gradient buffers"""
        model = {}
        model['W1'] = np.random.randn(nhidden, ninputs) / np.sqrt(ninputs)   # "Xavier" initialization
        model['W2'] = np.random.randn(nactions, nhidden) / np.sqrt(nhidden)
        self.model = model
        self.grad = { k : np.zeros_like(v) for k,v in model.iteritems() }
        self.gradsq = { k : np.zeros_like(v) for k,v in model.iteritems() }   
        

    def forward(self, s):
        """ Run the model forward given a state as input.
    returns action predictions and the hidden state"""
        h = np.dot(self.model['W1'], s)
        h[h<0] = 0 # ReLU nonlinearity
        rew = np.dot(self.model['W2'], h)
        return rew, h
    
    
    def predict(self, s):
        """ Predict the action rewards from a given input state"""
        rew, h = self.forward(s)
        return rew
    
              
    def gradient(self, s, a, y):
        """ Given a state s, action a and target y, compute the model gradients"""
    ##################################################################################
    ##                                                                              ##
    ## TODO: Compute gradients and return a scalar loss on a minibatch of size npar ##
    ##    s is the input state matrix (ninputs x npar).                             ##
    ##    a is an action vector (npar,).                                            ##
    ##    y is a vector of target values (npar,) corresponding to those actions.    ##
    ##    return: the loss per sample (npar,).                                      ##                          
    ##                                                                              ##
    ## Notes:                                                                       ##
    ##    * If the action is ai in [0,...,nactions-1], backprop only through the    ##
    ##      ai'th output.                                                           ##
    ##    * loss should be L2, and we recommend you normalize it to a per-input     ##
    ##      value, i.e. return L2(target,predition)/sqrt(npar).                     ##
    ##    * save the gradients in self.grad['W1'] and self.grad['W2'].              ##
    ##    * update self.grad['W1'] and self.grad['W2'] by adding the gradients, so  ##
    ##      that multiple gradient steps can be used beteween updates.              ##
    ##                                                                              ##
    ##################################################################################
        loss = 0.0 
        prediction, h = self.forward(s)
        diffs = np.zeros(npar)
        dW2 = np.zero_like(self.model['W2'])
        dW1 = np.zero_like(self.model['W1'])
        num_hidden = h.shape[0]
        for i in range(npar):
            si = s[:, i]
            ai = a[i]
            hi = h[:, i]
            pred_i = prediction[ai, i]
            residual = pred_i - y[i]
            diffs.append[i] = residual 
            dW2[ai, :] += 2 * residual * hi 
            dhidden = 2 * residual * self.model['W2']
            dhidden[hi == 0] = 0 
            dW1 += np.outer(dhidden, si)
        
        loss = np.linalg.norm(diffs, 2)
        constant = 1 / (2 * loss * np.sqrt(pars))
        dW1 *= constant
        dW2 *= constant
        
        self.grad['W1'] = dW1
        self.grad['W2'] = dW2

        return loss
    
    
    def rmsprop(self, learning_rate, decay_rate): 
        """ Perform model updates from the gradients using RMSprop"""
        for k in self.model:
            g = self.grad[k]
            self.gradsq[k] = decay_rate * self.gradsq[k] + (1 - decay_rate) * g*g
            self.model[k] -= learning_rate * g / (np.sqrt(self.gradsq[k]) + 1e-5)
            self.grad[k].fill(0.0)

In [ ]:
# Gradient Checking 


## 2. Implement $\epsilon$-Greedy Policy

An $\epsilon$-Greedy policy should:
* with probability $\epsilon$ take a uniformly-random action.
* otherwise choose the best action according to the estimator from the given state.

The function below should implement this policy. It should return a matrix A of size (nactions, npar) such that A[i,j] is the probability of taking action i on input j. The probabilities of non-optimal actions should be $\epsilon/{\rm nactions}$ and the probability of the best action should be $1-\epsilon+\epsilon/{\rm nactions}$.

Since the function processes batches of states, the input <code>state</code> is a <code>ninputs x npar</code> matrix, and the returned value should be a <code>nactions x npar</code> matrix. 

In [5]:
def policy(estimator, state, epsilon):
    """ Take an estimator and state and predict the best action.
    For each input state, return a vector of action probabilities according to an epsilon-greedy policy"""
    ##################################################################################
    ##                                                                              ##
    ## TODO: Implement an epsilon-greedy policy                                     ##
    ##       estimator: is the estimator to use (instance of Estimator)             ##
    ##       state is an (ninputs x npar) state matrix                              ##
    ##       epsilon is the scalar policy parameter                                 ##
    ## return: an (nactions x npar) matrix A where A[i,j] is the probability of     ##
    ##       taking action i on input j.                                            ##
    ##                                                                              ##
    ## Use the definition of epsilon-greedy from the cell above.                    ##
    ##                                                                              ##
    ##################################################################################
    predictions = estimator.predict(state)
    nactions, npar = np.shape(predictions)
    best_action = np.argmax(predictions, axis=1)
    A = np.zeros((nactions, npar))
    A += epsilon/nactions
    for i in range(npar):
        A[best_action[i], i] = 1 - epsilon + (epsilon / nactions)
    return A

This routine copies the state of one estimator into another. Its used to update the target estimator from the Q-estimator.

In [6]:
def update_estimator(to_estimator, from_estimator, window, istep):
    """ every <window> steps, Copy model state from from_estimator into to_estimator"""
    if (istep % window == 0):
        for k in from_estimator.model:
            np.copyto(to_estimator.model[k], from_estimator.model[k])

## 3. Implement "Asynchronous Threads"

Don't try that in Python!! Actually all we do here is create an array of environments and advance each one a random number of steps, using random actions at each step. Later on we will make *synchronous* updates to all the environments, but the environments (and their gradient updates) should remain uncorrelated. This serves the same goal as asynchronous updates in paper [2], or experience replay in paper [1].

In [9]:
import random
block_reward = 0.0;
total_epochs = 0;


block_reward = np.zeros((16), dtype=float);
total_epochs = np.zeros((16), dtype=float);
   
# Create estimators
q_estimator = Estimator(nfeats*nwindow, nhidden, nactions)
target_estimator = Estimator(nfeats*nwindow, nhidden, nactions)

# The epsilon and learning rate decay schedules
epsilons = np.linspace(epsilon_start, epsilon_end, neps)
learning_rates = np.linspace(learning_rate, lr_end, nlr)

# Initialize the games
print("Initializing games..."); sys.stdout.flush()
envs = np.empty(npar, dtype=object)
state = np.zeros([nfeats * nwindow, npar], dtype=float)
rewards = np.zeros([npar], dtype=float)
dones = np.empty(npar, dtype=int)
actions = np.zeros([npar], dtype=int)


for i in range(npar):
    envs[i] = gym.make(game_type)
   
    ##################################################################################
    ##                                                                              ##
    ## TODO: Advance each environment by a random number of steps, where the number ##
    ##       of steps is sampled uniformly from [nwindow, init_moves].              ##
    ##       Use random steps to advance.                                           ## 
    ##                                                                              ##
    ## Update the total reward and total epochs variables as you go.                ##
    ## If an environment returns done=True, reset it and increment the epoch count. ##
    ##                                                                              ##
    ##################################################################################
    length_game = random.randint(nwindow, init_moves)
    observation = envs[i].reset()
    for t in range(length_game):
        envs[i].render()
        actions[i] = envs[i].action_space.sample()
        print(actions[i])
        observation_prev = observation
        observation, rewards[i], dones[i], _ = envs[i].step(actions[i])
        print(rewards[i])
        block_reward[i] += rewards[i]
        total_epochs[i] += 1
        if dones[i]:
            print("Episode finished after {} timesteps".format(t+1))
            block_reward[i] = 0.0 # resettig total reward
            envs[i].reset()
    rewards[i] = block_reward[i]
    state[:, i] = np.hstack([observation, float(1), observation_prev, float(1)])
    

Initializing games...


[2016-11-21 16:52:26,941] Making new env: CartPole-v0


1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
Episode finished after 11 timesteps
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
Episode finished after 25 timesteps
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
Episode finished after 38 timesteps
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
Episode finished after 56 timesteps
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
Episode finished after 77 timesteps
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
Episode finished after 98 timesteps
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
Episode finished after 116 timesteps
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1

[2016-11-21 16:52:28,747] Making new env: CartPole-v0


0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
Episode finished after 1452 timesteps
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
Episode finished after 1475 timesteps
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
Episode finished after 1490 timesteps
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
Episode finished after 1516 timesteps
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
Episode finished after 20 timesteps
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1


...

1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
Episode finished after 11 timesteps
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
Episode finished after 26 timesteps
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
Episode finished after 37 timesteps
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
Episode finished after 48 timesteps
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
Episode finished after 74 timesteps
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
Episode finished after 92 timesteps
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
Episode finished after 114 timesteps
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1

[2016-11-21 16:55:19,609] Making new env: CartPole-v0


1
1.0
0
1.0
0
1.0
0
1.0
Episode finished after 1539 timesteps
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
Episode finished after 1591 timesteps
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
Episode finished after 1602 timesteps
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
Episode finished after 1643 timesteps
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
Episode finished after 1670 timesteps


[2016-11-21 16:55:28,922] Making new env: CartPole-v0


1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
Episode finished after 20 timesteps
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
Episode finished after 34 timesteps
1
1.0
1
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
Episode finished after 62 timesteps
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
Episode finished after 83 timesteps
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
1
1.0
1
1.0
1
1.0
1
1.0
Episode finished after 105 timesteps
1
1.0
1
1.0
0
1.0
1
1.0
0
1.0
1
1.0
1
1.0
0
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
0
1.0
0
1.0
1
1.0
1
1.0
1
1.0
Epi

## 4. Implement Deep Q-Learning
In this cell you actually implement the algorithm. We've given you comments to define all the steps. You should also add book-keeping steps to keep track of the loss, reward and number of epochs (where env.step() returns done = true). 

In [ ]:
t0 = time.time()
block_loss = 0.0
last_epochs=0

for istep in np.arange(nsteps): 
    if (render): envs[0].render()
  
    #########################################################################
    ## TODO: Implement Q-Learning                                          ##
    ##                                                                     ##
    ## At high level, your code should:                                    ##
    ## * Update epsilon and learning rate.                                 ##
    ## * Update target estimator from Q-estimator if needed.               ##
    ## * Get the next action probabilities for the minibatch by running    ##
    ##   the policy on the current state with the Q-estimator.             ##
    ## * Then for each environment:                                        ##
    ##     ** Pick an action according to the action probabilities.        ##
    ##     ** Step in the gym with that action.                            ##
    ##     ** Process the observation and concat it to the last nwindow-1  ##
    ##        processed observations to form a new state.                  ##
    ## Then for all environments (vectorized):                             ##
    ## * Predict Q-scores for the new state using the target estimator.    ##
    ## * Compute new expected rewards using those Q-scores.                ##
    ## * Using those expected rewards as a target, compute gradients and   ##
    ##   update the Q-estimator.                                           ##
    ## * Step to the new state.                                            ##
    ##                                                                     ##
    #########################################################################
    current_eps_index = int(istep / (nstep / (1.0*neps)))
    current_eps = epsilons[current_eps_index]
    current_lr = learning_rates[current_eps_index]
    
    update_estimator(q_estimator, target_estimator, window, istep)
    next_action_probs = policy(estimator, state, epsilon)
    
    
    
   

    t = time.time() - t0
    if (istep % printsteps == 0): 
        print("step {:0d}, time {:.1f}, loss {:.8f}, epochs {:0d}, reward/epoch {:.5f}".format(
                istep, t, block_loss/printsteps, total_epochs, block_reward/np.maximum(1,total_epochs-last_epochs)))
        last_epochs = total_epochs
        block_reward = 0.0
        block_loss = 0.0

Let's save the model now. 

In [ ]:
pickle.dump(q_estimator.model, open("cartpole_q_estimator.p", "wb"))


You can reload the model later if needed:

In [ ]:
test_estimator = Estimator(nfeats*nwindow, nhidden, nactions)
test_estimator.model = pickle.load(open("cartpole_q_estimator.p", "rb"))

And animate the model's performance. 

In [ ]:
state0 = state[:,0]
for i in np.arange(200):
    envs[0].render()
    preds = test_estimator.predict(state0)
    iaction = np.argmax(preds)
    obs, _, done0, _ = envs[0].step(VALID_ACTIONS[iaction])
    state0 = np.concatenate((state0[nfeats:], preprocess(obs)))
    if (done0): envs[0].reset()
    

So there we have it. Simple 1-step Q-Learning can solve easy problems very fast. Note that environments that produce images will be much slower to train on than environments (like CartPole) which return an observation of the state of the system. But this model can still train on those image-based games - like Atari games. It will take hours-days however. It you try training on visual environments, we recommend you run the most expensive step - rmsprop - less often (e.g. every 10 iterations). This gives about a 3x speedup. 

## Optional
Do **one** of the following tasks:
* Adapt the DQN algorithm to another environment - it can use direct state observations.  Call <code>env.get_action_meanings()</code> to find out what actions are allowed. Summarize training performance: your final average reward/epoch, the number of steps required to train, and any modifications to the model or its parameters that you made.
* Try smarter schedules for epsilon and learning rate. Rewards for CartPole increase very sharply (several orders of magnitude) with better policies, especially as epsilon --> 0. Gradients will also change drastically, so the initial learning rate is probably not good later on. Try schedules for decreasing epsilon that allow the model to better adapt. Try other learning rate schedules, or setting learning rate based on average reward. 
* Try a fancier model. e.g. add another hidden layer, or try sigmoid non-linearities.